In [78]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
from tqdm import trange
from operator import itemgetter
import numpy as np
from pprint import pprint

import matplotlib.pyplot as plt
from matplotlib import rc
gridc = (1., 1., 1)
plt.rcParams['grid.color'] = gridc
plt.rcParams["axes.edgecolor"] = (0.898, 0.925, 0.965, 1)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

from database import DataBase

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
db = DataBase('assets/neurips2021.db')
db.initialize()

### statistics

In [92]:
# consistency experiment
_cmd = "SELECT * FROM submissions WHERE num_decision == 2 ORDER BY title;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
print("> Total submissions with CE: {}".format(len(data) // 2))
num_comflict = 0
num_consist = 0
ces = {}
for i in range(0, len(data), 2):  
    dcs = data[i][6].split(' ')[-1].strip('()')
    dcs_0 = data[i][7].split(' ')[-1].strip('()')
    dcs_1 = data[i + 1][7].split(' ')[-1].strip('()')  
    _key = ", ".join(sorted([dcs_0, dcs_1]))
    _key = dcs + ', ' + _key
    if _key in ces.keys():
        ces[_key] += 1
    else:
        ces.update({_key: 1})
    assert data[i][2] == data[i + 1][2], f'not match for {i}'
    if dcs_0 != dcs_1:
        num_comflict += 1
    else:
        num_consist += 1
print(f"> Decision conflict: {num_comflict}")
print(f"> Decision consist: {num_consist}")
pprint(ces)

> Total submissions with CE: 298
> Decision conflict: 221
> Decision consist: 77
{'Oral, Oral, Poster': 5,
 'Poster, Poster, Poster': 74,
 'Poster, Poster, Reject': 174,
 'Poster, Poster, Spotlight': 4,
 'Poster, Reject, Spotlight': 16,
 'Spotlight, Oral, Poster': 1,
 'Spotlight, Poster, Spotlight': 12,
 'Spotlight, Reject, Spotlight': 9,
 'Spotlight, Spotlight, Spotlight': 3}


### write consistency experiment

In [62]:
_cmd = "SELECT * FROM submissions WHERE num_decision == 2 " \
       "ORDER BY CASE final_decision WHEN 'Accept (oral)' " \
       "THEN 'a' WHEN 'Accept (Spotlight)' THEN 'b' WHEN 'Accept (Poster)' " \
       "THEN 'c' WHEN 'Reject' THEN 'd' END, title DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

# read template 
with open('neurips2021_consis_exp_template.html', 'r') as f:
    html_temp = f.readlines()
# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1

# write data
for i in trange(len(data)):
    _data = data[i]
    final_d = _data[6].split(' ')[-1].strip('()')
    now_d = _data[7].split(' ')[-1].strip('()')
    _str = f"<tr><td>{i}</td><td class='td-left'><a href='{_data[1]}'> {_data[2]}</a></td>" \
           f"<td>{_data[9]:.2f}</td><td>{_data[10]:.2f}</td><td data-sort='{_data[8]}'>{_data[11]}</td>" \
           f"<td class='{final_d.lower()}' data-sort='{maps[final_d]}'>{final_d}</td>" \
           f"<td class='{now_d.lower()}' data-sort='{maps[now_d]}'>{now_d}</td>" \
           f"<td class='th-rank'>{check}</td></tr>\n"
    html_temp.insert(idx + i, _str)

with open('neurips2021_consistency_experiments.html' ,'w') as f:
    f.write("".join(html_temp))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 596/596 [00:00<00:00, 319056.18it/s]


### write all submissions

In [61]:
check = '&#10004;'
maps = {
    'Oral': 'a',
    'Spotlight': 'b',
    'Poster': 'c',
    'Reject': 'd'
}

In [64]:
_cmd = "SELECT * FROM submissions ORDER BY rating_avg DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

# read template 
with open('neurips2021_submissions_template.html', 'r') as f:
    html_temp = f.readlines()
# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1

# write data
for i in trange(len(data)):
    _data = data[i]
    final_d = _data[6].split(' ')[-1].strip('()')
    now_d = _data[7].split(' ')[-1].strip('()')
    _str = f"<tr><td>{i}</td><td class='td-left'><a href='{_data[1]}'> {_data[2]}</a></td>" \
           f"<td>{_data[9]:.2f}</td><td>{_data[10]:.2f}</td><td data-sort='{_data[8]}'>{_data[11]}</td>" \
           f"<td class='{final_d.lower()}' data-sort='{maps[final_d]}'>{final_d}</td>" \
           f"<td class='{now_d.lower()}' data-sort='{maps[now_d]}'>{now_d}</td>" \
           f"<td class='th-rank'>{check if _data[5] == 2 else ''}</td></tr>\n"
    html_temp.insert(idx + i, _str)

with open('neurips2021_submissions.html' ,'w') as f:
    f.write("".join(html_temp))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 2632/2632 [00:00<00:00, 428116.35it/s]


In [57]:
float(_data[10])

0.82915619758885

In [50]:
f"{float(6.567):.2f}"

'6.57'

In [35]:
"(asdf)".strip("()")

'asdf'

In [18]:
data[1]

(24,
 'https://openreview.net/forum?id=UVQNdLIELSU',
 'Replacing Rewards with Examples: Example-Based Policy Search via Recursive Classification',
 'reinforcement learning, example-based control',
 'Benjamin Eysenbach, Sergey Levine, Ruslan Salakhutdinov',
 2,
 'Accept (Oral)',
 'Accept (Oral)',
 4,
 8.0,
 0.7071067811865476,
 '8, 7, 9, 8')

In [9]:
db.cursor.execute("select * from submissions where ")

OperationalError: incomplete input

In [10]:
2632 - (596 / 2)

2334.0

In [ ]:

metadata = MetaData('assets/data')

In [ ]:
sorted_idxs = np.argsort(metadata.get_all_values_by_key('avg_score'))[::-1]
avg_scores = metadata.get_all_values_by_key('avg_score')
avg_scores = itemgetter(*sorted_idxs)(avg_scores)
titles = metadata.get_all_values_by_key('title')
titles = itemgetter(*sorted_idxs)(titles)
urls = metadata.get_all_values_by_key('url')
urls = itemgetter(*sorted_idxs)(urls)
scores = metadata.get_all_values_by_key('scores')
scores = itemgetter(*sorted_idxs)(scores)
keywords = metadata.get_all_values_by_key('keywords')
keywords = itemgetter(*sorted_idxs)(keywords)

In [ ]:
hist, bin_edges = np.histogram(avg_scores, bins=20, range=(0, 10))
mean_score = np.mean(avg_scores)

In [ ]:
width = 0.4
fig = plt.figure(figsize=[16, 6])

ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor((0.898, 0.925, 0.965, 0.5))
ax.spines['left'].set_color('w')
ax.spines['bottom'].set_color('w')
ax.spines['right'].set_color('w')
ax.spines['top'].set_color('w')

ax.bar(np.linspace(0, 10, len(hist)), hist, width=width, alpha=0.95, 
       color='#789BFF', capsize=4)

for i, v in zip(np.linspace(0, 10, len(hist)), hist):
    ax.text(i - 0.2, v + 3.5, str(v), color='#1f59ff', fontsize=16)

ax.text(0, 400, f"Average rating: {mean_score:.1f}", color='#1f59ff', fontsize=16)
plt.ylim(0, 680)
plt.xticks(ticks=np.linspace(0, 10, len(hist)), rotation=40, labels=[f"{d:.1f}" for d in np.linspace(0, 10, len(hist))])
ax.set_ylabel(r"# submissions", fontsize=14)
ax.set_xlabel("Rating", fontsize=14)
ax.set_axisbelow(True)
ax.grid()
plt.savefig('images/stats_bar.png', dpi=300)
